In [1]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta, time
from IPython.display import display

# for jupyt notebooks
import os
d = os.path.dirname(os.path.dirname(os.getcwd()))
print(d)
import sys
sys.path.append(d)
from ATJ.mt5_trade_utils.backtester import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

c:\Users\derne\OneDrive - The Pennsylvania State University\Programming\Extra\algobot


In [2]:
symbol = 'XAUUSDm'
start_dt = datetime(2023, 1, 1)
end_dt = datetime.now()

# get historical data
mt5.initialize()
ohlc = get_ohlc_history(symbol, mt5.TIMEFRAME_H2, start_dt, end_dt)

sma_period = 20
ohlc['sma_20'] = ohlc['open'].rolling(sma_period).mean()

display(ohlc)

fig = create_price_fig(ohlc, indicators=['sma_20'])
display(fig)

,time,open,high,low,close,sma_20
0,2023-01-02 22:00:00,1826.216,1830.460,1823.998,1829.131,NaN
1,2023-01-03 00:00:00,1829.110,1832.330,1826.747,1828.477,NaN
2,2023-01-03 02:00:00,1828.510,1843.079,1828.260,1841.153,NaN
3,2023-01-03 04:00:00,1841.196,1842.066,1837.936,1840.337,NaN
4,2023-01-03 06:00:00,1840.377,1849.902,1839.986,1847.464,NaN
...,...,...,...,...,...,...
6365,2025-01-24 12:00:00,2776.671,2784.615,2776.094,2782.382,2757.88790
6366,2025-01-24 14:00:00,2782.418,2786.013,2773.723,2775.973,2759.21205
6367,2025-01-24 16:00:00,2775.918,2777.600,2769.894,2773.312,2760.22985
6368,2025-01-24 18:00:00,2773.337,2775.277,2771.325,2774.628,2761.25980


Strategy

- buy when price opens above sma_20
- sell when price opens below sma_20


In [4]:
def get_signal(x):
    if x['open'] > x['sma_20']:
        return 'buy'
    elif x['open'] < x['sma_20']:
        return 'sell'
    else:
        return ''
    
ohlc['signal'] = ohlc.apply(get_signal, axis=1)
ohlc

,time,open,high,low,close,sma_20,signal
0,2023-01-02 22:00:00,1826.216,1830.460,1823.998,1829.131,NaN,
1,2023-01-03 00:00:00,1829.110,1832.330,1826.747,1828.477,NaN,
2,2023-01-03 02:00:00,1828.510,1843.079,1828.260,1841.153,NaN,
3,2023-01-03 04:00:00,1841.196,1842.066,1837.936,1840.337,NaN,
4,2023-01-03 06:00:00,1840.377,1849.902,1839.986,1847.464,NaN,
...,...,...,...,...,...,...,...
6365,2025-01-24 12:00:00,2776.671,2784.615,2776.094,2782.382,2757.88790,buy
6366,2025-01-24 14:00:00,2782.418,2786.013,2773.723,2775.973,2759.21205,buy
6367,2025-01-24 16:00:00,2775.918,2777.600,2769.894,2773.312,2760.22985,buy
6368,2025-01-24 18:00:00,2773.337,2775.277,2771.325,2774.628,2761.25980,buy


In [8]:
# create trade logic
def on_bar(data, trades, orders):
    volume = 100000 # 1 lots
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['signal'] == 'buy' and not num_open_trades:
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        orders.open_trade(symbol, volume, 'sell')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data['signal'] == 'sell':
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell' and data['signal'] == 'buy':
            orders.close_trade(trade)

In [9]:
# backtest parameters
starting_balance = 1000
currency = 'USD'
exchange_rate = 1
commission = -7 / 100000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,XAUUSDm,buy,100000,2023-01-04 12:00:00,1860.137,2023-01-05 08:00:00,1849.305,0,0,{},-1083200.0,-7.0,-1083207.0,-1083207.0,-1082207.0
1,closed,XAUUSDm,sell,100000,2023-01-05 10:00:00,1848.491,2023-01-06 14:00:00,1851.79,0,0,{},-329900.0,-7.0,-329907.0,-1413114.0,-1412114.0
2,closed,XAUUSDm,buy,100000,2023-01-06 16:00:00,1862.284,2023-01-10 08:00:00,1871.957,0,0,{},967300.0,-7.0,967293.0,-445821.0,-444821.0
3,closed,XAUUSDm,buy,100000,2023-01-10 10:00:00,1876.799,2023-01-10 14:00:00,1873.297,0,0,{},-350200.0,-7.0,-350207.0,-796028.0,-795028.0
4,closed,XAUUSDm,buy,100000,2023-01-10 16:00:00,1875.702,2023-01-10 18:00:00,1874.217,0,0,{},-148500.0,-7.0,-148507.0,-944535.0,-943535.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,closed,XAUUSDm,sell,100000,2025-01-20 14:00:00,2705.041,2025-01-20 16:00:00,2707.063,0,0,{},-202200.0,-7.0,-202207.0,45568915.0,45569915.0
555,closed,XAUUSDm,buy,100000,2025-01-20 18:00:00,2707.441,2025-01-23 04:00:00,2749.995,0,0,{},4255400.0,-7.0,4255393.0,49824308.0,49825308.0
556,closed,XAUUSDm,buy,100000,2025-01-23 06:00:00,2751.97,2025-01-23 10:00:00,2745.793,0,0,{},-617700.0,-7.0,-617707.0,49206601.0,49207601.0
557,closed,XAUUSDm,sell,100000,2025-01-23 12:00:00,2751.873,2025-01-23 18:00:00,2757.747,0,0,{},-587400.0,-7.0,-587407.0,48619194.0,48620194.0


In [10]:
pnl_chart = bt.plot_pnl()
pnl_chart

In [11]:
backtest_fig = bt.visualize_backtest(indicators=['sma_20'])
backtest_fig

In [ ]:
bt.export_to_json('sma_backtest.json', indicators=['sma_20'])

In [ ]:
evaluate_backtest(bt.trades)